# Librerías a utilizar

In [69]:
from base64 import b32encode
from dotenv import load_dotenv
from funciones_auxiliares import obtener_access_token_para_la_api_de_spotify
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests

# Análisis exploratorio de los datos

## Metadata

In [70]:
track_features = pd.read_csv('data/tf_mini.csv')
print('Dataset Track Features:')
print('- Filas:', track_features.shape[0])
print('- Columnas:', track_features.shape[1])
print('- Datos nulos:', track_features.isna().sum().sum())

Dataset Track Features:
- Filas: 50704
- Columnas: 30
- Datos nulos: 0


In [71]:
sessions = pd.read_csv('data/log_mini.csv')
print('Dataset Sessions:')
print('- Filas:', sessions.shape[0])
print('- Columnas:', sessions.shape[1])
print('- Datos nulos:', sessions.isna().sum().sum())


Dataset Sessions:
- Filas: 167880
- Columnas: 21
- Datos nulos: 0


## Agregar columnas a Track Features

In [72]:
load_dotenv()
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
access_token = obtener_access_token_para_la_api_de_spotify(client_id, client_secret)
access_token

'BQB3YvieWZ0vFE5K8_1l5BWTmxgjoyHyVz-U5CV70qJlh8qByBvhNqilLPuUhbsPsB7CCB3WmufwauIbWEjSIFaiN73ZRIuvRPQ00s5YPxA8J7JESt0'

In [73]:
track_features.tail()

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
50699,t_402930af-4174-47ec-b1fd-593d93597624,184.686798,2018,99.315966,0.584765,0.521544,0.515087,0.653140,7.684220,0.336433,...,4,0.542063,-0.196001,0.301727,0.238880,-0.391421,0.016690,0.247235,-0.399387,-0.192473
50700,t_e5f9a069-a893-452e-ab21-49b4eaebfbd0,251.813324,2018,99.918573,0.406680,0.565200,0.693126,0.844861,11.176841,0.709085,...,4,0.472353,-0.545160,0.271596,0.274377,0.043951,-0.322946,0.150802,0.159378,0.384336
50701,t_3983306d-13b4-4027-9391-7236ca93d2bf,157.520004,2018,98.517692,0.001279,0.414721,0.341769,0.463543,5.405471,0.975503,...,4,0.766519,0.112592,0.368523,-0.466950,-0.468494,0.640088,0.050771,-0.258999,0.258766
50702,t_74eb6e99-210b-440c-8d7b-4db6617d1c80,129.105392,2018,99.902866,0.139452,0.688375,0.733720,0.850959,10.778521,0.666146,...,4,0.058505,-0.855291,0.365487,0.273034,0.108294,-0.206204,0.007847,-0.408226,0.143629
50703,t_aec0dda3-19b4-4222-be07-327c12f21456,213.689392,2018,99.961653,0.154936,0.327645,0.365520,0.520514,6.707502,0.880616,...,4,0.414714,-0.095364,0.382398,0.231202,-0.419750,-0.010569,0.173470,-0.403821,-0.377302


In [80]:
track_id = str(b32encode('t_aec0dda3-19b4-4222-be07-327c12f21456'.encode()))[2:-1]
print(track_id)
track = requests.get(
    url=f'https://api.spotify.com/v1/tracks/{track_id}',
    headers={
        'Authorization': 'Bearer ' + access_token,
        'Content-Type': 'application/json'
    }
)
print(track.text)

ORPWCZLDGBSGIYJTFUYTSYRUFU2DEMRSFVRGKMBXFUZTEN3DGEZGMMRRGQ2TM===
{
  "error" : {
    "status" : 400,
    "message" : "invalid id"
  }
}
